In [13]:
import numpy as np
import pandas as pd

In [14]:
data1 = pd.read_csv( 'train.csv')

data2 = pd.read_csv( 'comments.csv')

data3 = pd.read_csv( 'comments2.csv')

data4 = pd.read_csv( 'ar_reviews_100k.csv', delimiter='\t')

data = pd.concat( ( data1, data2, data3, data4)).reset_index(drop=True)

import sklearn
data = sklearn.utils.shuffle( data)

data = data.drop_duplicates(subset='comment', keep='first')

X = data[ 'comment']

In [15]:
print( X.shape)

(104655,)


In [16]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop = set(stopwords.words('arabic'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aminb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
import string

punctuation = list(string.punctuation)
stop.update(punctuation)

In [18]:
import re
from clean import wordopt

def split_into_words(text):
    words = text.split()
    return words

def remove_punctuation(words):
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    stripped = [re_punc.sub('', w) for w in words]
    return stripped

def remove_stopwords(words):
    stop_words = set(stopwords.words('arabic'))
    words = [w for w in words if not w in stop_words]
    return words

def to_sentence(words):
    return ' '.join(words)

def denoise_text(text):
    text = wordopt( text)
    words = split_into_words(text)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return words

In [19]:
vocabulary = X.apply( denoise_text)

In [20]:
X

16054            جيد. شكرا على حسن تعاونكم. الواي فاي ضعيف
64368    لطيفة بس كل ما أحاول أستمتع بيها وأستحضر شكل و...
92845    لم اتوقع هذا علي الاطلاق دخلت الفيلم لأري عبقر...
85375         لا تحكم ع كتاب من غلافه .. خاب ظني كثيرا : .
9251     كتاب مفيد جدا لدرجة تجعلك تتمناه في جيبك كل ال...
                               ...                        
7025     بصراحه تجربة تستحق التكرار مرة اخرى . الشقة كا...
81281    مخيب للأمل. الموقع وسرعة الباصات. خدمات البوفي...
82692               ثرثرة حمقاء. ورواية قوية. في نفس الوقت
54634    لابأس به . موقع جيد. الفندق لايستحق نجوم في رأ...
99183    مخيب للأمل. . السعر أعلى من حجم وامكانيات الفن...
Name: comment, Length: 104655, dtype: object

In [21]:
vocabulary

16054            [جيد, شكرا, حسن, تعاونكم, واي, فاي, ضعيف]
64368    [لطيفة, احاول, استمتع, بيها, واستحضر, شكل, ورو...
92845    [اتوقع, علي, اطلاق, دخلت, فيلم, لاري, عبقرية, ...
85375                        [تحكم, كتاب, غلافه, خاب, ظني]
9251     [كتاب, مفيد, جدا, لدرجة, تجعلك, تتمناه, جيبك, ...
                               ...                        
7025     [بصراحه, تجربة, تستحق, تكرار, مرة, اخرى, شقة, ...
81281    [مخيب, امل, موقع, وسرعة, باصات, خدمات, بوفيه, ...
82692                     [ثرثرة, حمقا, ورواية, قوية, وقت]
54634    [لاباس, موقع, جيد, فندق, لايستحق, نجوم, راي, ع...
99183    [مخيب, امل, سعر, اعلى, حجم, وامكانيات, فندقالف...
Name: comment, Length: 104655, dtype: object

In [22]:
vocabulary.loc[0]

['اوصي',
 'منبر',
 'توجه',
 'مراكز',
 'تلقيح',
 'صدقوني',
 'امر',
 'مزحة',
 'مرض',
 'زكام',
 'بالفعل',
 'مرض',
 'قاتل',
 'ايضا',
 'كنت',
 'يريدون',
 'لقاح',
 'عندما',
 'اصيب',
 'زوج',
 'اختى',
 'وشاهدت',
 'حالته',
 'وصلت',
 'موت',
 'لكف',
 'الله',
 'لكان',
 'مرحومين',
 'وعمره',
 'ثلاثينلا',
 'تصدقوا',
 'شائعات',
 'حول',
 'مرض',
 'وانه',
 'قاتل',
 'ويصيب',
 'فقط',
 'عواجيز',
 'كله',
 'كذب',
 'يصيبىاي',
 'احد',
 'ويقتل',
 'اشد',
 'ناس',
 'قوةاتمنى',
 'مغاربة',
 'يقدموا',
 'مصلحة',
 'مجتمع',
 'انانيتهم',
 'وكفى',
 'نشر',
 'ترهات',
 'وسخافات']

In [23]:
import gensim

w2v_model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4,
)

In [24]:
w2v_model.build_vocab( vocabulary, progress_per=1000)

In [26]:
w2v_model.train( vocabulary, total_examples=w2v_model.corpus_count, epochs=100)

(382521676, 405687800)

In [27]:
w2v_model.save("./word2vec-arabic.model")

In [28]:
w2v_model.wv.most_similar("لقاح")

[('تلقيح', 0.8557243347167969),
 ('لقاحات', 0.7613672018051147),
 ('قاح', 0.6941816210746765),
 ('ملقحين', 0.6669356822967529),
 ('ملقح', 0.6631062030792236),
 ('فيروس', 0.6630459427833557),
 ('باللقاح', 0.6499715447425842),
 ('بالتلقيح', 0.6332268118858337),
 ('جرعتين', 0.6255894899368286),
 ('قاحات', 0.6170222163200378)]

In [29]:
w2v_model.wv.similarity(w1="لقاح", w2="مرض")

0.23614615

In [30]:
w2v_model.wv.similarity(w1="لقاح", w2="حرية")

0.16686115